In [7]:
import numpy as np
import pandas as pd
from nltk import sent_tokenize
import re
import pymorphy2
import string
from tqdm import tqdm_notebook
from sklearn.metrics.pairwise import cosine_similarity
import operator
from collections import Counter
regex = re.compile('[%s]' % re.escape(string.punctuation))

In [8]:
morph = pymorphy2.MorphAnalyzer()

In [9]:
def get_model(data, mode):
    if mode == 'collection':
        tokens = ' '.join(data).split()
    elif mode == 'doc':
        tokens = data.split()
    else:
        raise ValueError('invalid mode')
    collection_length = len(tokens)
    collection_model = Counter(tokens)
    for key in collection_model.keys():
        collection_model[key] /= collection_length
    return collection_model

In [10]:
def get_query_prob(query, lambda_coef, collection_model, doc_model):
    p = 1
    valid_prob = False
    for token in query.split():
        if token in collection_model:
            valid_prob = True
            collection_prob = collection_model[token]
            if token in doc_model:
                doc_prob = doc_model[token]
            else:
                doc_prob = collection_prob
            p *= ((1 - lambda_coef) * collection_prob + lambda_coef * doc_prob)
    if valid_prob:
        return p
    return 0

In [11]:
facts = ['Калифорнийский посёлок был независим от США почти 3 месяца пока не пришла пора праздновать День независимости и закупать алкоголь',
        'В наполеоновской артиллерии служили три генерала из одной семьи',
        'Совершивший первое кругосветное путешествие на велосипеде ездил по России на коне']
normalized_facts = []
for i in range(len(facts)):
    tokens = facts[i].split()
    for j in range(len(tokens)):
        tokens[j] = morph.parse(tokens[j])[0].normal_form
    normalized_facts.append(' '.join(tokens))

## Normalized facts 

In [12]:
for fact in normalized_facts:
    print(fact, '\n')

калифорнийский посёлок быть независимый от сша почти 3 месяц пока не пришлый пора праздновать день независимость и закупать алкоголь 

в наполеоновский артиллерия служить три генерал из один семья 

совершить первое кругосветный путешествие на велосипед ездить по россия на кон 



In [13]:
docs = open('data.txt', encoding='utf8').read()
docs = sent_tokenize(docs)
normalized_docs = []
for i in range(len(docs)):
    docs[i] = regex.sub(' ', docs[i])
    docs[i] = docs[i][:-1]
    tokens = docs[i].split()
    for j in range(len(tokens)):
        tokens[j] = morph.parse(tokens[j])[0].normal_form     
    tokens_without_stop_words = []
    stop_POS = ['PREP', 'CONJ', 'PRCL', 'NPRO']
    for token in tokens:
        POS = morph.parse(token)[0].tag.POS
        if POS not in stop_POS and POS is not None:
            tokens_without_stop_words.append(token)
    normalized_docs.append(' '.join(tokens_without_stop_words))

In [14]:
collection_model = get_model(normalized_docs, 'collection')
doc_models = [get_model(doc, 'doc') for doc in normalized_docs]

# Fact 1
Калифорнийский посёлок был независим от США почти 3 месяца пока не пришла пора праздновать День независимости и закупать алкоголь

In [28]:
probs = {}
lambda_coef = 0.5
for i in range(len(doc_models)):
    probs[i] = get_query_prob(normalized_facts[0], lambda_coef, collection_model, doc_models[i])

top_sentences_index = np.array(sorted(probs.items(), key=operator.itemgetter(1), reverse=True)[:5])[:, 0].astype(int)
for index in top_sentences_index:
    print(docs[index], '\n-----------\n')

Почтовая служба США требовала от жителей для упрощения своей работы сменить название их посёлка на Раф или Рэди но те отказались 
-----------

В телесериале Дни в Долине Смерти о независимости и скорой обратной зависимости Раф энд Рэди рассказывается в двух эпизодах  Rough and Ready    и Birthright    
-----------

Из достопримечательностей в Раф энд Рэди остались три здания х годов постройки ежегодно апреля отмечается собственный день независимости 
-----------

Независимое государство с апреля по июля года 
-----------

Более того в ближайшем городе Невада Сити старателям отказались продавать алкоголь так как они иностранцы 
-----------



# Fact 2
В наполеоновской артиллерии служили три генерала из одной семьи

In [29]:
probs = {}
lambda_coef = 0.5
for i in range(len(doc_models)):
    probs[i] = get_query_prob(normalized_facts[1], lambda_coef, collection_model, doc_models[i])

top_sentences_index = np.array(sorted(probs.items(), key=operator.itemgetter(1), reverse=True)[:5])[:, 0].astype(int)
for index in top_sentences_index:
    print(docs[index], '\n-----------\n')

Оба его сына были генералами артиллеристами Наполеоновской армии 
-----------

Первый генеральный инспектор артиллерии 
-----------

Во время Ста дней все три представителя семьи Д’Абовиль отказались активно поддерживать Наполеона 
-----------

Семья Д’Абовилей существует до сих пор и является одной из самых многочисленных среди сохранившихся семей французской знати 
-----------

Огюстен Габриэль Д’Абовиль французский военачальник эпохи Наполеоновских войн бригадный генерал военный артиллерист 
-----------



# Fact 3
Совершивший первое кругосветное путешествие на велосипеде ездил по России на коне

In [30]:
probs = {}
lambda_coef = 0.5
for i in range(len(doc_models)):
    probs[i] = get_query_prob(normalized_facts[2], lambda_coef, collection_model, doc_models[i])

top_sentences_index = np.array(sorted(probs.items(), key=operator.itemgetter(1), reverse=True)[:5])[:, 0].astype(int)
for index in top_sentences_index:
    print(docs[index], '\n-----------\n')

Томас Стивенс  человек совершивший первое кругосветное путешествие на велосипеде проехав  миль и пробыв в пути более двух лет  с апреля по декабрь года  
-----------

На всём протяжении своего кругосветного путешествия Стивенс отправлял свои путевые заметки в Harper’s Magazine которые позже вышли в двухтомнике с страницами Вокруг света на велосипеде 
-----------

Первое трансатлантическое путешествие завершилось августа года продлившись всего  дня без учёта вынужденных остановок из за плохой погоды и прочих неурядиц 
-----------

Позже Томас устроился на работу в шахту в Колорадо где ему пришла идея о путешествии на велосипеде по стране 
-----------

Оставив велосипед в подземном хранилище Стивенс поездом добрался до Лондона чтобы договориться об условиях своего путешествия в Европе и Азии 
-----------

